In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from prettytable import PrettyTable
import pandas as pd
from sklearn import preprocessing
import numpy as np
import time
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.neighbors import KNeighborsRegressor

In [ ]:
import json
import math

# Loading data

In [ ]:
headers = ["point","resource_type","n_units","n_affected","n_injured","mag"]

data = open('/content/drive/MyDrive/DM Project 2022/requests.json')
j = json.load(data)
# data = data.reset_index(drop=True)
# data = data.iloc[1:]
# data.describe()

In [ ]:
resource = pd.read_json('/content/drive/MyDrive/DM Project 2022/resources.json')
resource.head(10)

,points,resource_type,n_units
0,"[20.469655744850257, 71.7334380496474]",2,3
1,"[20.297787281472225, 73.49011792284956]",2,19
2,"[23.53742672439629, 72.87523907998961]",1,11
3,"[22.752063653820958, 72.49564711035343]",2,18
4,"[22.868343837736262, 71.65472183512952]",1,10
5,"[23.29938025347925, 73.89951162207002]",2,9
6,"[19.266014665987232, 73.04191646273532]",3,7
7,"[20.206575334827868, 73.94133654851272]",2,6
8,"[19.06650984158717, 72.44592271116173]",2,19
9,"[21.689997594278424, 73.77814563335642]",3,7


In [ ]:
resource.columns

Index(['points', 'resource_type', 'n_units'], dtype='object')

In [ ]:
df = pd.read_json('/content/drive/MyDrive/DM Project 2022/requests.json')
df.head(10)

,points,resource_type,n_units,n_affected,n_injured,mag
0,"[20.382334371474645, 72.73083096956852]",3,8,978,529,6.598650
1,"[19.724997202240537, 72.92216478691205]",2,12,2572,199,4.658472
2,"[19.71039071475183, 72.90355470636263]",2,18,1084,844,6.451535
3,"[19.974553247897756, 73.00542981635876]",2,14,1568,1502,7.571238
4,"[20.56565501693188, 73.21851002254664]",1,1,2250,1398,5.829998
5,"[21.661441643661647, 73.05646000198685]",1,12,3254,821,6.035564
6,"[20.63870294439276, 72.85336647042142]",3,16,2911,571,4.722908
7,"[22.592953421203543, 72.59413482659238]",2,4,1948,624,6.867365
8,"[22.617677651999635, 72.74543355865633]",2,1,4119,520,7.794920
9,"[20.34527870020502, 72.43660534429605]",2,11,2442,2241,5.668217


## Computing priority based on no. of people affected, no. of people injured and magnitude

In [ ]:
df1 = df.copy()

In [ ]:
df1.shape

(1999, 6)

In [ ]:
df1 = df1.to_numpy()

In [ ]:
df1[0]

array([list([20.382334371474645, 72.73083096956852]), 3, 8, 978, 529,
       6.598649860509344], dtype=object)

In [ ]:
# give weights to each parameter as 
# n_affected = 0.3
# n_injured = 0.4
# mag = 0.3


# Update table, drop n_affected, n_injured and mag


# for i in range(0,df1.shape[0]):
#   print(df1[i])

priority = []
def compute_priority(df1):
  for i in range(0,df1.shape[0]):
    p = df1[i][3] * 0.3 + df1[i][4] * 0.4 + df1[i][5] * 0.3
    # return (data - np.min(data)) / (np.max(data) - np.min(data))
    priority.append(p)
    # print(df1[i])

In [ ]:
compute_priority(df1)

In [ ]:
df2 = pd.DataFrame(df1, columns = headers)

In [ ]:
len(priority)

1999

In [ ]:
df2['priority'] = priority

In [ ]:
df2.columns

Index(['point', 'resource_type', 'n_units', 'n_affected', 'n_injured', 'mag',
       'priority'],
      dtype='object')

In [ ]:
df2 = df2.drop(['n_affected','n_injured','mag'], axis = 1)

In [ ]:
def normalize_priority(df2):
  df2['priority'] = (df2['priority'] - np.min(df2['priority'])) / (np.max(df2['priority']) - np.min(df2['priority']))
  return df2


In [ ]:
df2 = normalize_priority(df2)

In [ ]:
df2.head()

,point,resource_type,n_units,priority
0,"[20.382334371474645, 72.73083096956852]",3,8,0.148053
1,"[19.724997202240537, 72.92216478691205]",2,12,0.249546
2,"[19.71039071475183, 72.90355470636263]",2,18,0.194379
3,"[19.974553247897756, 73.00542981635876]",2,14,0.314408
4,"[20.56565501693188, 73.21851002254664]",1,1,0.362121


# Implementation

In [ ]:
# sort dataset on priority and for each entry, find a warehouse
df3 = df2.sort_values(by=['priority'], ascending=False)

In [ ]:
# add a column initilized to 0 to resources
resource['allocated'] = 0

In [ ]:
resource['idx'] = [i for i in range(0,resource.shape[0])]


In [ ]:
resource

,points,resource_type,n_units,allocated,idx
0,"[20.469655744850257, 71.7334380496474]",2,3,0,0
1,"[20.297787281472225, 73.49011792284956]",2,19,0,1
2,"[23.53742672439629, 72.87523907998961]",1,11,0,2
3,"[22.752063653820958, 72.49564711035343]",2,18,0,3
4,"[22.868343837736262, 71.65472183512952]",1,10,0,4
...,...,...,...,...,...
4994,"[22.768260957604983, 71.9274291050135]",3,5,0,4994
4995,"[20.840004281217812, 73.12154471233369]",1,8,0,4995
4996,"[21.297067861806575, 74.16291560901378]",2,1,0,4996
4997,"[19.53089597923881, 71.44443195255619]",1,14,0,4997


In [ ]:
df3.head(20)

,point,resource_type,n_units,priority
626,"[20.94027261154515, 73.15601108659867]",3,8,1.000000
1991,"[21.467516128420144, 72.43088750556925]",3,16,0.994610
1955,"[19.782688037201932, 72.96197841993066]",1,13,0.993952
487,"[21.287092875097585, 73.24625449558239]",3,1,0.978862
1928,"[20.014603828775837, 72.49523420764473]",2,8,0.956089
1532,"[21.687907333504203, 73.20057104475252]",2,7,0.944376
1475,"[21.625607673809984, 72.7016249327647]",2,12,0.939410
76,"[22.142782456569613, 72.85946064670429]",2,15,0.938685
451,"[20.652952642915455, 72.73815698199819]",1,15,0.938436
947,"[21.42268706234714, 72.66157805465761]",1,15,0.935460


In [ ]:
# Implement algorithm - sort on priority, pick one row and find a warehouse

In [ ]:
def allocate_warehouse(resource, datapoint):
  
  re = resource[resource['resource_type'] == datapoint[1]]
  re = resource[resource['n_units'] >= datapoint[2]]
  # re = resource[resource[1] == datapoint[1]]
  # re = resource[resource[2] >= datapoint[2]]
  
  re = re.to_numpy()
  neighbours = []

  for i in range(0,re.shape[0]):
    dist = 0
    dist = (re[i][0][0] - datapoint[0][0]) ** 2 + (re[i][0][1] - datapoint[0][1]) ** 2
    neighbours.append((dist,re[i][4]))

  neighbours.sort(key=lambda y: y[0])
  if len(neighbours) == 0:
    return "No resources available"
  return neighbours[0]

    # find eucleadean distance, sort and then see which can fulfill the request, if none can optimze it later to allocate partially from different warehouse
    # match resource type




In [ ]:
df3 = df3.to_numpy()

In [ ]:
df3[4]

array([list([20.014603828775837, 72.49523420764473]), 2, 8,
       0.9560894192049225], dtype=object)

In [ ]:
nearest_neighbour = allocate_warehouse(resource, df3[4])

In [ ]:
nearest_neighbour[1]

275

In [ ]:
# resources_available = df3

In [ ]:
resource

,points,resource_type,n_units,allocated,idx
0,"[20.469655744850257, 71.7334380496474]",2,3,0,0
1,"[20.297787281472225, 73.49011792284956]",2,19,0,1
2,"[23.53742672439629, 72.87523907998961]",1,11,0,2
3,"[22.752063653820958, 72.49564711035343]",2,18,0,3
4,"[22.868343837736262, 71.65472183512952]",1,10,0,4
...,...,...,...,...,...
4994,"[22.768260957604983, 71.9274291050135]",3,5,0,4994
4995,"[20.840004281217812, 73.12154471233369]",1,8,0,4995
4996,"[21.297067861806575, 74.16291560901378]",2,1,0,4996
4997,"[19.53089597923881, 71.44443195255619]",1,14,0,4997


In [ ]:
# resource = resource.to_numpy()

In [ ]:
# resource[nearest_neighbour[1]][2] = resource[nearest_neighbour[1]][2] - df3[4][2]
# resource[nearest_neighbour[1]][3] = resource[nearest_neighbour[1]][3] + df3[4][2]

In [ ]:
# resource[nearest_neighbour[1]][2]


In [ ]:
resources_header = ['points', 'resource_type', 'n_units', 'allocated', 'idx']

In [ ]:
# resource = resource.to_numpy()

In [ ]:
def fulfill_requests(resource):
  for i in range(0,df3.shape[0]):
    # print("case 1")
    nearest_neighbour = allocate_warehouse(resource, df3[i])
    print(nearest_neighbour)
    if nearest_neighbour == "No resources available":
      continue
    resource = resource.to_numpy()
    resource[nearest_neighbour[1]][2] = resource[nearest_neighbour[1]][2] - df3[i][2]
    resource[nearest_neighbour[1]][3] = resource[nearest_neighbour[1]][3] + df3[i][2]
    resource = pd.DataFrame(resource, columns = resources_header)
    # df2 = pd.DataFrame(df1, columns = headers)
  return resource


In [ ]:
df3.shape

(1999, 4)

In [ ]:
# resources_available[nearest_neighbour[1]][2] = resources_available[nearest_neighbour[1]][2] - df3[4][2]
# resources_available[nearest_neighbour[1]][4] = resources_available[nearest_neighbour[1]][4] + df3[4][2]

In [ ]:
# def fulfill_requests():
#   for i in range(0,df3.shape[0]):
#     nearest_neighbour = allocate_warehouse(resource, df3[i])
#     resources_available[nearest_neighbour[1]][2] = resources_available[nearest_neighbour[1]][2] - df3[i][2]
#     resources_available[nearest_neighbour[1]][4] = resources_available[nearest_neighbour[1]][4] + df3[i][2]


In [ ]:
resource_map = fulfill_requests(resource)

(0.0009037906473715482, 2708)
(0.007382852637488437, 849)
(0.005030273132666863, 3904)
(0.00014367567591490863, 3332)
(0.00031943608398781484, 275)
(0.00018345508436287575, 3149)
(0.0018922814015553328, 4820)
(0.009707819282352393, 2958)
(0.0017797423193493009, 1498)
(0.0006886934870772101, 1238)
(0.0014576906946701762, 2123)
(0.0031550467052879148, 477)
(0.0031340823552782364, 688)
(0.0010934556308435962, 535)
(0.0003209436636212049, 2420)
(0.001856044651858431, 3272)
(0.0027850991665418507, 81)
(0.0036654265693848733, 2268)
(0.0067600151612925135, 2346)
(0.001679962858514481, 685)
(0.001414462110922088, 2503)
(0.0012203323392387588, 2440)
(0.00035627641973989225, 2885)
(0.0005718417193010078, 4560)
(0.0013840736282090171, 325)
(0.0007876156986213848, 1119)
(3.912909228140484e-05, 945)
(0.004216822774720567, 2920)
(0.001579040633601311, 4787)
(0.0011556090965091283, 4214)
(0.00012054911392936438, 463)
(9.259885588639859e-05, 1975)
(0.0012427780801603217, 179)
(0.0012259642193629494, 1

# Requested vs Allocated

In [ ]:
resource_map

,points,resource_type,n_units,allocated,idx
0,"[20.469655744850257, 71.7334380496474]",2,3,0,0
1,"[20.297787281472225, 73.49011792284956]",2,0,19,1
2,"[23.53742672439629, 72.87523907998961]",1,11,0,2
3,"[22.752063653820958, 72.49564711035343]",2,3,15,3
4,"[22.868343837736262, 71.65472183512952]",1,10,0,4
...,...,...,...,...,...
4994,"[22.768260957604983, 71.9274291050135]",3,5,0,4994
4995,"[20.840004281217812, 73.12154471233369]",1,1,7,4995
4996,"[21.297067861806575, 74.16291560901378]",2,1,0,4996
4997,"[19.53089597923881, 71.44443195255619]",1,14,0,4997


In [ ]:
resource_map = resource_map.to_numpy()

check if there are any negative values

In [ ]:
for i in range(0,len(resource_map)):
  if resource_map[i][3] < 0:
    print('Huh')

In [ ]:
for i in range(0,len(resource_map)):
  if resource_map[i][3] < 0:
    print('Huh')

In [ ]:
resource_map = pd.DataFrame(resource_map, columns = resources_header)

In [ ]:
resource_map['Total_resources'] = resource_map['n_units'] + resource_map['allocated']

In [ ]:
resource_map

,points,resource_type,n_units,allocated,idx,Total_resources
0,"[20.469655744850257, 71.7334380496474]",2,3,0,0,3
1,"[20.297787281472225, 73.49011792284956]",2,0,19,1,19
2,"[23.53742672439629, 72.87523907998961]",1,11,0,2,11
3,"[22.752063653820958, 72.49564711035343]",2,3,15,3,18
4,"[22.868343837736262, 71.65472183512952]",1,10,0,4,10
...,...,...,...,...,...,...
4994,"[22.768260957604983, 71.9274291050135]",3,5,0,4994,5
4995,"[20.840004281217812, 73.12154471233369]",1,1,7,4995,8
4996,"[21.297067861806575, 74.16291560901378]",2,1,0,4996,1
4997,"[19.53089597923881, 71.44443195255619]",1,14,0,4997,14


In [ ]:
# plt.title("train loss vs epochs for different optimizer") 
# plt.xlabel("Epoch")
# plt.ylabel("Loss") 
# plt.plot(x,y1 ,'y')
# plt.plot(x,y2 ,'g') 
# plt.plot(x,y3 ,'b') 
# plt.plot(x,y4 ,'p')
# plt.show()

In [ ]:
import matplotlib.pyplot as plt
 
  
# # creating the dataset
# data = {'C':20, 'C++':15, 'Java':30,
#         'Python':35}
total = list(resource_map['Total_resources'])
allocated = list(resource_map['allocated'])
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(total, allocated, color ='maroon',
        width = 0.4)
 
plt.xlabel("Allocated Resources")
plt.ylabel("Total Resources")
plt.title("Total vs Allocated")
plt.show()